<a href="https://colab.research.google.com/github/NiloyKumarKundu/Natural-Language-Processing/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Datasets/fake news classification.csv')

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
df = df.dropna()

In [31]:
X = df.drop('label', axis=1)
y = df['label']

In [32]:
X.shape

(18285, 4)

In [33]:
y.shape

(18285,)

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [35]:
voc_size = 5000

In [36]:
# One hot encoding

messages = X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [37]:
messages['title'][2]

'Why the Truth Might Get You Fired'

In [38]:
messages.reset_index(inplace=True)

In [39]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [41]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [42]:
one_hot_rep = [one_hot(words, voc_size) for words in corpus]
one_hot_rep

[[3288, 3636, 3422, 4089, 2696, 2822, 635, 4932, 3297, 1900],
 [540, 2027, 1665, 3813, 1254, 4932, 3151],
 [2931, 553, 473, 354],
 [4118, 4235, 1751, 3137, 1587, 1235],
 [2787, 1254, 2286, 3009, 2693, 1851, 1254, 675, 4592, 4287],
 [4627,
  1975,
  1001,
  4310,
  3841,
  273,
  2107,
  1611,
  2145,
  4471,
  2422,
  3063,
  2882,
  4049,
  3151],
 [871, 1290, 1417, 4858, 3959, 3477, 2608, 2669, 2629, 2868, 1192],
 [3730, 2204, 1068, 4548, 4765, 3951, 273, 1156, 2629, 2868, 1192],
 [837, 4993, 2804, 3656, 4170, 2435, 4278, 4367, 273, 3929],
 [3922, 4946, 3553, 4922, 98, 1975, 4331, 2004],
 [2076, 4473, 1249, 3634, 682, 1714, 1319, 1751, 1255, 2443, 1083],
 [3137, 765, 2696, 2435, 273, 4765],
 [2562, 4104, 244, 2306, 1766, 400, 3289, 4551, 399],
 [4061, 3511, 2525, 3918, 680, 3631, 4973, 2629, 2868, 1192],
 [2057, 447, 4967, 748, 2037, 2629, 2868, 1192],
 [4381, 2811, 3286, 1759, 1574, 3831, 95, 1742, 2591, 311],
 [894, 2838, 2027],
 [1293, 4368, 3538, 2899, 273, 3064, 745, 3151],
 [37

# Embedding

In [43]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4932 3297 1900]
 [   0    0    0 ... 1254 4932 3151]
 [   0    0    0 ...  553  473  354]
 ...
 [   0    0    0 ... 2629 2868 1192]
 [   0    0    0 ... 2566  135 4027]
 [   0    0    0 ... 4320 2374 2436]]


In [44]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3288,
       3636, 3422, 4089, 2696, 2822,  635, 4932, 3297, 1900], dtype=int32)

In [45]:
# Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [46]:
len(embedded_docs), y.shape

(18285, (18285,))

In [47]:
X_final = np.array(embedded_docs)
y_final = np.array(y)
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [49]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8154 - loss: 0.4196 - val_accuracy: 0.9125 - val_loss: 0.1972
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9442 - loss: 0.1444 - val_accuracy: 0.9171 - val_loss: 0.1938
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9691 - loss: 0.0941 - val_accuracy: 0.9201 - val_loss: 0.2057
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9749 - loss: 0.0700 - val_accuracy: 0.9027 - val_loss: 0.2550
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9866 - loss: 0.0476 - val_accuracy: 0.9150 - val_loss: 0.2801
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9914 - loss: 0.0281 - val_accuracy: 0.9130 - val_loss: 0.3768
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9958 - loss: 0.0171 - val_accuracy: 0.9125 - val_loss: 0.4270
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9970 - loss: 0.0119 - val_accuracy: 0

In [50]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [51]:
from sklearn.metrics import confusion_matrix

# Convert predictions to binary values based on a threshold
threshold = 0.5  # Example threshold - adjust as needed
y_pred_binary = (y_pred > threshold).astype(int)

# Calculate confusion matrix with binary predictions
confusion_matrix(y_test, y_pred_binary)

array([[3107,  312],
       [ 240, 2376]])

In [52]:
from sklearn.metrics import accuracy_score

# Convert predictions to binary values based on a threshold
threshold = 0.5  # Example threshold - adjust as needed
y_pred_binary = (y_pred > threshold).astype(int)

# Calculate accuracy with binary predictions
accuracy_score(y_test, y_pred_binary)

0.9085335542667772